Forrás:
https://www.openml.org/search?type=data&sort=runs&id=13&status=active  
https://github.com/datasets/breast-cancer/blob/master/data/breast-cancer.csv

In [ ]:
# Standard könyvtárak
import numpy as np

# Harmadik féltől származó könyvtárak
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.svm import SVC


In [ ]:

# Beolvasás a CSV fájlból pandas DataFrame-be
df = pd.read_json('bc.json')

# Az első néhány sor megjelenítése, hogy ellenőrizzük a beolvasás sikerességét
df.head()

In [ ]:
# Hiányzó értékek meghatározása az adathalmazban
missing_values = df.isnull().sum()

# Hiányzó értékek számának megjelenítése
print(missing_values)

# Sorok eldobása, ahol bármilyen hiányzó érték van
df_cleaned = df.dropna()

# Ellenőrizzük újra a hiányzó értékeket, hogy megbizonyosodjunk azok eltávolításáról
missing_values_cleaned = df_cleaned.isnull().sum()

# A tisztított adathalmaz első néhány sorának megjelenítése
df_cleaned.head(), missing_values_cleaned

### Plotok
Készítsük el a következő plotokat:
- Korrelációs heatmap
- Violin plot a 'deg-malig' változó eloszlásának vizualizálása különböző 'class' csoportokban
- Count plot az érintett mell ('breast') gyakoriságának ábrázolására

In [ ]:
# Az adatok numerikus változóira koncentrálunk a korrelációs mátrix létrehozásához
# Mivel az adathalmaz többnyire kategórikus adatokat tartalmaz, először át kell alakítanunk ezeket numerikussá
df_numerical = pd.get_dummies(df_cleaned)

# 1. Korrelációs mátrix készítése
corr_matrix = df_numerical.corr()

# Korrelációs heatmap rajzolása
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, cmap='coolwarm', annot=False)
plt.title('Korrelációs Heatmap')
plt.show()

# 2. Violin plot 
plt.figure(figsize=(8, 6))
sns.violinplot(x='class', y='deg-malig', data=df_cleaned)
plt.title('Deg-Malig Eloszlás Class Csoportok Szerint')
plt.show()

# 3. Count plot
plt.figure(figsize=(6, 4))
sns.countplot(x='breast', data=df_cleaned)
plt.title('Érintett Mell Gyakorisága')
plt.show()

In [ ]:
# Nem numerikus (kategórikus) oszlopok meghatározása
non_numeric_columns = df_cleaned.select_dtypes(include=['object']).columns

non_numeric_columns


In [ ]:
# Előkészítés: Életkor konvertálása int-é az átlag alapján
df_cleaned['age'] = df_cleaned['age'].apply(lambda x: int(np.mean([int(n) for n in x.split('-')])))
# Tumor méret és inv-nodes konvertálása hasonlóan
df_cleaned['tumor-size'] = df_cleaned['tumor-size'].apply(lambda x: int(np.mean([int(n) for n in x.split('-')])))
df_cleaned['inv-falsedes'] = df_cleaned['inv-falsedes'].apply(lambda x: int(np.mean([int(n) for n in x.split('-')])))

# Label Encoding a 'breast', 'class', és 'breast-quad' oszlopokon
label_encoder = LabelEncoder()
df_cleaned['breast'] = label_encoder.fit_transform(df_cleaned['breast'])
df_cleaned['class'] = label_encoder.fit_transform(df_cleaned['class'])
df_cleaned['breast-quad'] = label_encoder.fit_transform(df_cleaned['breast-quad'])

# One-Hot Encoding a 'falsede-caps', 'deg-malig', és 'mefalsepause' oszlopokon
one_hot_columns = ['falsede-caps', 'deg-malig', 'mefalsepause']
df_cleaned = pd.get_dummies(df_cleaned, columns=one_hot_columns)

# Skálázás az 'age' oszlopon
scaler_age = MinMaxScaler(feature_range=(-1, 1))
df_cleaned['age'] = scaler_age.fit_transform(df_cleaned[['age']])

# Skálázás a 'tumor-size' és 'inv-falsedes' oszlopokon
scaler_tumor_inv = MinMaxScaler(feature_range=(-1, 1))
df_cleaned['tumor-size'] = scaler_tumor_inv.fit_transform(df_cleaned[['tumor-size']])
df_cleaned['inv-falsedes'] = scaler_tumor_inv.fit_transform(df_cleaned[['inv-falsedes']])

# A DataFrame első néhány sorának megjelenítése az előkészítés után
df_cleaned.head()


In [ ]:
# Új korrelációs mátrix készítése az előkészített adathalmazból
corr_matrix_updated = df_cleaned.corr()

# Korrelációs heatmap rajzolása az új adatokkal
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix_updated, cmap='coolwarm', annot=False)
plt.title('Korrelációs Heatmap az Előkészített Adatokkal')
plt.show()


In [ ]:

# A célváltozó (class) és a magyarázó változók (feature-ök) különválasztása
X = df_cleaned.drop('class', axis=1)  # Minden oszlop kivéve a 'class'
y = df_cleaned['class']  # Csak a 'class' oszlop

# Az adatok szétválasztása tanító és teszt halmazra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

# A tanító és teszt halmazok méretének ellenőrzése
(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


In [ ]:


# SVM modell létrehozása
svm_model = SVC(kernel='linear', random_state=42)

# Modell betanítása a tanító adathalmazon
svm_model.fit(X_train, y_train)

# A modell tesztelése a teszt adathalmazon
y_pred = svm_model.predict(X_test)

# Teljesítményértékelés
accuracy = accuracy_score(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(accuracy)
print(class_report)


In [ ]:

# Confusion matrix létrehozása
conf_matrix = confusion_matrix(y_test, y_pred)

# Confusion matrix megjelenítése heatmap segítségével
plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', square=True)
plt.title('Confusion Matrix')
plt.ylabel('Tényleges osztály')
plt.xlabel('Jósolt osztály')
plt.show()
